# Combining Image and Tabular Data in a predictive model

Revisiting a clasic idea: Suppose I am trying to build a model that predicts the price of a house. Earlier in the course, we looked at this problem using tabulated data and a variety of different regression models.

![](regression.png)

In this example, the regression model could be any of a number of different models, such as linear regression, random forest, boosted trees, etc.

Suppose that,instead of having a table of house features as our `X` matrix, we have image data. We could use a CNN model to make our predictor:

![](cnn.png)

Here, the CNN can be as simple or as complicated as we like, but the general structure is to use Convo2D and pooling to decompose the image into a set of features. These features are then flattened to form a vector which more or less summarizes the salient features of the house. These features are then the inputs to dense layers which feed an output layer which provides our final prediction.

Suppose we now want to create a more advanced model that combines these two different types of data, tabular data and image data to make better predictions.

It's possible (but not advisable) to simply append the tabulated data values to the image data. Here the numeric values from the data table are treated as extra pixels in the image.

![](simple_hybrid.png)

I don't think this is a good approach because the nature of the convolution allows pixels to shift left and right (and up and down). This would essentially scramble the meaning of these features. I.e, a house with 4 bedrooms, 3 bathrooms and 1200 square feet would be treated the same as a house with 3 bedrooms and 1200 bathrooms, which is not going to be very sensible. We need to ensure we are using convolution on the image, but not on the tabulated features.  

There are a number of possible approaches to do this.

The simplest approach might be to make a meta model which takes as its inputs the predictions from the two models we have already built.
![](meta_model1.png)

The two separate sets of results could be combined into a single data frame using code such as:

```
hybrid_df['prediction_1'] = regression_model.predict(X)
hybrid_df['prediction_2'] = CNN_model.predict(image)

meta_model.predict(hybrid_df)
```

Bearing in mind, of course, to maintain correspondence between the two prediction models so that results from one house's features are not aligned with results from another house's image.


While this model has the advantage of combing information both tabular and image, with appropriate treatement for each type, it is somewhat limited in how it can combine that data. The final prediction might not be any more complex than a simple weighted average between the two predictions.

Rather than combine the data at the output of the initial predictors, another approach would be to treate the prediction from the CNN model as an additional feature of the tabulated data.

![](meta_model2.png)

Rather than create a new data frame from the initial predictions, we append the CNN prediction using code such as:

```
df['CNN_prediction'] = CNN_model.predict(image)

meta_model.predict(df)
```

or:

```
meta_model.predict(pd.concat([df, CNN_model.predict(image)], axis = 1]))
```

This approach allows for greater interaction between the features than simply looking at the final predictions.

Perhaps a drawback of these last two approaches is that they over-simplify the information from the image. Here, the entire image has been simplified down to just a single value (a prediction of price.) Rather than just use this one value as the only output from the CNN model, we could get a richer set of features derived from the image. Fortunately, the CNN allows us to get these values from one of the flat layers prior to the final output.

![](meta_model3.png)

The additional features, `CNN_featurex` don't have an interpretation as simple as the original features (bedrooms, bathrooms, square feet), but they are features nonetheless which may be informative for our final model. They can be thought of as digested components of the image.

Note that we are not using the predicted output from the CNN model to make our ultimate prediction (although we could, if we wanted, add that predicted value in as yet another feature column). Nevertheless, we need to train our CNN model to some target, so the predicted price from the CNN is needed at during training.

Another approach would be to replace the CNN prediction model with an autoencoder trained to decompose and then recompose images. The interior of the autoencoder (bottleneck) is a vector of features which describes the house and can be used either to reconstruct an image of the house, or to feed a final model.

![](meta_model4.png)

Here the image provides the target needed to train the autoencoder and the bottleneck layer provides the vector of encoded image features which are appeneded to the orginal set of features to make the final prediction.

As we saw in the autoencoder sprint, tensorflow allows models to be built from other models, just as from layers, so this gives us the ability to break the autoencoder into multiple parts and use the encoder portion both to feed the decoder and and to generate its own output vector.

The keras Functional API  `layers.concatenate()` method allows us to join two different vectors inside a keras model. So this process can be done either with a keras model into a pandas data frame, into an sklearn model, or entirely inside of a keras model.